# Assignment 5

***

*Smart Contracts and Decentralized Applications*

***

Now that we have completed the essential functionalities of our blockchain, let's look at an application case that is probably the most promising: **decentralized applications**.

In concrete terms, if we summarize blockchain to its most pure principle, it is a decentralized and immutable data structure over time. And in fact, **decentralized** and **immutable over time** are the key points that make decentralized applications an interesting model.

The objective of this Assignment 5 is to lay the foundations of smart contracts, which are a future technological support for all decentralized applications. It is also mainly the basis that will probably serve for your mini-project: it will be based either on the mechanics of smart contracts or on another mechanic that you will implement from scratch on the foundations of new types of certificates.

In [100]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


For the entirety of Assignment 5, we will need a blockchain. The following cell defines a function `reset_blockchain()` that creates/recreates the following setup, which you have already roughly had in Assignment 4:
* 3 nodes on a network
* Each node stakes in the 2nd block: 1 token for node 1, 2 for node 2, and 3 for node 3

You will have nothing else to do but to send certificates directly to one of the nodes using the function `new_certificate(certificate)`, which is also the interest of having completed Assignment 4 before this one.

Feel free to use this function `reset_blockchain()`. There will come cases where you change the source code of your smart contracts while the blockchain still exists, and this may pose a problem because we are not going to implement the functionality to update contracts.

In [101]:
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

walletAlice = Wallet()

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)

### Smart contracts

With what we've covered in the course, you are now capable of implementing smart contracts in your blockchain. Here's a reminder:
* Smart contracts are created by embedding a piece of code in a certificate, which is then added to the blockchain.
* To modify one or more values of the contract, you must create a new certificate that will engrave our contract modification action into the blockchain.
* Reading the values of a contract takes into account all the modification certificates present in the blockchain.

We will implement these three functionalities. For this, it is important to distinguish two types of certificates: contract definition certificates, and contract writing certificates.

For contract definition, we need a way to store the source code of the contract but also to do something with it. In the Ethereum blockchain, for example, smart contracts are stored in their compiled form (with 0s and 1s). In our situation, we are lucky to benefit from the many practical aspects of Python's "interpreted language" characteristic: its ability to be executed *on the fly*.

> For your knowledge: there are two types of programming languages: the "compiled" type, where your code is converted into 0s and 1s before execution (for example, when you create a .exe file, the code has already been compiled) and the "interpreted" type, where compilation occurs at the time of execution, and where the reading of the code is done "in reading order". Python is of the "interpreted" type.

Therefore, we will code our contracts in Python. **A Smart Contract will thus be represented by a string describing a Python class named `SmartContract`.** Here are the set of rules we will follow when coding a smart contract:
* The contract's constructor always takes the creator's public key as a parameter.
* Any contract function intended to modify the state of the contract (by changing one or more of its values) must have the public key of the person calling this function as the first argument.
* Other functions are read functions.

***

<font color="7777aa">In a new file `smart_contract.py`, still in your `scripts` folder, create the class `SmartContractDefinition`. Since it's a certificate, this class must inherit from `Certificate`, and you need to replace the `build_payload()` function to incorporate its `sourceCode` field.

`sourceCode` is a string that contains the source code of the Smart Contract.</font>

> To illustrate our smart contract implementation, we will be building a minimal chat application. In the next cell, you can observe how such a decentralized application can be built.

In [102]:
from smart_contract import SmartContractDefinition

dummySmartContract = '''

class SmartContract:
    
    def __init__(self, ownerPublicKey):
        self.messages = []
        
    def write_message(self, callerPublicKey, message):
        displayedName = callerPublicKey[256:264]
        self.messages.append(f'[{displayedName}] {message}')
        
    def display_chat(self):
        for message in self.messages:
            print(message)

'''

dummySmartContractDefinition = SmartContractDefinition(walletAlice.publicKey, dummySmartContract)
walletAlice.sign(dummySmartContractDefinition)

assert dummySmartContractDefinition.__class__.__bases__[-1].__name__ == "Certificate"
assert dummySmartContractDefinition.sourceCode == dummySmartContract
assert dummySmartContract in dummySmartContractDefinition.build_payload().values()

"Success!"

'Success!'

Having the source code isn't enough for our blockchain to manipulate the smart contract. We need to link this code to a purely Python object that can be manipulated from the Python environment in which our blockchain resides.

To achieve this, we need to add a function to our class that converts the source code into a Python object. Fortunately for us, Python has a built-in function `exec(pythonCode)` which, literally, executes the Python code `pythonCode` as if you had written it verbatim in place of `exec(pythonCode)`.

There's just one subtlety: the variables you generate in this code are not directly accessible. You must go through the `locals()` object, which contains a mapping of all local variable names to their corresponding object. Here is an example:

In [103]:
def showcase_exec():
    exec('pi = 3.14159', locals=locals())
    print(locals())

    try:
        print(pi)
    except:
        print('"pi" is unknown')

    print(f"You need to go through \"locals()['pi']\": {locals()['pi']}")

<font color="7777aa">Following this example, implement in `SmartContractDefinition` the function `instantiate_contract()` which generates a Python object from the contract's source code.</font>

> <details><summary><strong>Click here for help</strong></summary>When you declare a class in Python, what you're actually doing is creating a variable that has the same name as the class and contains a reference to the type generated by the class. To call its constructor, you simply need to add parentheses <code>()</code> behind what you get out of <code>locals()</code>.</details>

In [104]:
from smart_contract import SmartContractDefinition

smartContractObject = dummySmartContractDefinition.instantiate_contract()
assert 'messages' in smartContractObject.__dict__
assert len(smartContractObject.messages) == 0

smartContractObject.write_message(walletAlice.publicKey, "I'm a smart contract")

assert len(smartContractObject.messages) == 1

"Success!"

Instantiated contract: <smart_contract.SmartContract object at 0x000002405B9DF7D0>


'Success!'

Now that we have a way to store the source code of a smart contract in the blockchain, we need a way to modify its values.

It's actually very simple, in our example just above, if Alice wants to call the function `write_message(message)` of the contract to add a message like `"Hello!"`, she has two options:
* Generate an instance of the contract using the `instantiate_contract()` function, then call `write_message("Hello!")` on it.
* Create a certificate that states **"I, Alice, call the function `write_message` with the parameter `"Hello!"`."**

While the first of the two options seems to be the most enticing because it is quick and easy, it is absolutely not valid, as the function call happens **locally, on Alice's machine**. The other people in the blockchain will have no knowledge of the contract value modification.

On the other hand, in the 2nd case, since everyone has access to the blockchain, everyone will know that Alice has called the function `write_message(message)` with the parameter `"Hello!"`. Therefore, to have the true updated value of the contract, they will follow Alice's directives and perform the call `write_message("Hello!")` on their side.

***

<font color="7777aa">Still in `smart_contract.py`, following your `SmartContractDefinition` class, create a 2nd class `SmartContractWritingOperation` that also inherits from `Certificate` and contains the fields:
* `targetSmartContractHash` which contains the address (the hash) of the contract in question
* `targetFunctionName` which contains the name of the function to call
* `functionArgumentList` which contains the arguments to use in the function call

Again, do not forget to rewrite the `build_payload()` function.</font>

In [105]:
from smart_contract import SmartContractWritingOperation

dummySmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'write_message',
    ["Bonjour!"]
)

walletAlice.sign(dummySmartContractWritingOperation)

assert dummySmartContractWritingOperation.__class__.__bases__[-1].__name__ == "Certificate"
assert dummySmartContractWritingOperation.targetSmartContractHash == dummySmartContractDefinition.hash()
assert dummySmartContractWritingOperation.targetFunctionName == 'write_message'
assert dummySmartContractWritingOperation.functionArgumentList[0] == "Bonjour!"

"Success!"

'Success!'

Thanks to Python's "interpreted" nature, there's a simple way to call a function of an object via its name and a list of arguments. Two tools allow you to do this:
* The function `getattr(object, name)` returns the attribute of the object `object` named `name`. For example, `getattr(dummyCertificate, 'issuerPublicKey')` will return the value of `issuerPublicKey` from the certificate `dummyCertificate`. If `name` designates a function, `getattr` will return the function, which you can then call by placing parentheses `()` after it.
* The asterisk `*` "explodes" collections. Since it's a bit complicated to explain with words, let's show you an example: if you have a list of integers `myList = [0, 1, 2]` and any function `dummy_function(a, b, c)` that expects 3 parameters as input, calling `dummy_function(*myList)` is perfectly valid, because `*` will "explode" the list into three items `0`, `1`, and `2` in sequence.

Here's a cell that shows you how to call the function `append(element)` of a Python list using this system.

In [106]:
myList = ["Baldur's Gate 3"]

listAppendFunction = getattr(myList, 'append')
listAppendFunction(*['GOTY'])

myList

["Baldur's Gate 3", 'GOTY']

It's important to understand the difference between the two types of certificates we've created in this Assignment 5:
- Certificates of type `SmartContractDefinition` are responsible for keeping the source code of smart contracts in memory on the blockchain. They also allow creating an instance of the smart contract as a manipulable Python object.
- Certificates of type `SmartContractWritingOperation`, on the other hand, record changes in the state of the smart contract in the blockchain. Therefore, they must be used in conjunction with `SmartContractDefinition`: by making a change directly on the Python object generated by the `SmartContractDefinition`.

***

<font color="7777aa">Write in `SmartContractWritingOperation` the function `apply_on_contract(contractPythonObject)` which, like the example above, executes the function `targetFunctionName` with the arguments `functionArgumentList` on the contract `contract`.</font>

In [107]:
smartContractObject = dummySmartContractDefinition.instantiate_contract()
dummySmartContractWritingOperation.apply_on_contract(smartContractObject)

assert smartContractObject.messages[0][11:] == "Bonjour!"

"Success!"

Instantiated contract: <smart_contract.SmartContract object at 0x000002405BA2FE30>


'Success!'

We're almost done implementing smart contracts in our blockchain. There's just one last function to code, which will allow us to compute, at the current moment, the complete state of a smart contract. The process is as follows:
- For a given smart contract (identified by its hash), we will traverse the entire blockchain in ascending order until we come across the `SmartContractDefinition` that declares it. We thus create a Python object from it.
- From there, we continue to traverse the blockchain, and for each `SmartContractWritingOperation` we find along the way that concerns this smart contract, we will apply the modification conveyed by this `SmartContractWritingOperation`.
- At the end of the traversal, we will have a Python instance of the smart contract that is as up-to-date as possible.

***

<font color="7777aa">In your `SmartContractDefinition` class, implement the **static** function `get_smart_contract_at_current_state(blockchain, targetSmartContractHash)` that returns an instance of the smart contract in its current state according to the blockchain.

This function must generate an instance of the contract and then apply to it, one by one, the writing operations concerning it found in the blockchain, to obtain its current state.
    
> **ATTENTION**: This time, the `timestamp` field proves useful because it is absolutely necessary to execute the writing operations in their order of issuance. Use `sorted(itemList, key=lambda x: x.timestamp)` to sort them chronologically.

***

Even if your function is correct, you will not pass the assert. Why? Complete the cell so that your blockchain has a sufficient state to take into account Alice's modification on the smart contract.</font>

In [108]:
from certificate import Certificate

reset_blockchain()

dummySmartContractDefinition = SmartContractDefinition(walletAlice.publicKey, dummySmartContract)
walletAlice.sign(dummySmartContractDefinition)

dummySmartContractWritingOperation = SmartContractWritingOperation(
    walletAlice.publicKey,
    dummySmartContractDefinition.hash(),
    'write_message',
    ['Bonjour!']
)

walletAlice.sign(dummySmartContractWritingOperation)

node1.new_certificate(dummySmartContractDefinition)
node1.new_certificate(dummySmartContractWritingOperation)

######################


#add your code here

# The blockchain node is set up so that it only creates a new block once it has collected 5 certificates.
# In our example, we only added two certificates (the smart contract definition and the writing operation). 
# Without a block, these certificates aren’t committed to the blockchainn !
# so, b y adding three additional (dummy) certificates, we reach the required count.


dummyCertificate1 = Certificate(walletAlice.publicKey)
walletAlice.sign(dummyCertificate1)
node1.new_certificate(dummyCertificate1)

dummyCertificate2 = Certificate(walletAlice.publicKey)
walletAlice.sign(dummyCertificate2)
node1.new_certificate(dummyCertificate2)

dummyCertificate3 = Certificate(walletAlice.publicKey)
walletAlice.sign(dummyCertificate3)
node1.new_certificate(dummyCertificate3)


######################

smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node2.blockchain,
    dummySmartContractDefinition.hash()
)

assert smartContractObject.messages[0][11:] == "Bonjour!"

"Success!"

Instantiated contract: <smart_contract.SmartContract object at 0x000002405B62AB10>


'Success!'

### Mini-project: Decentralized Application

With what you've just implemented, you've understood: smart contracts are nothing more than a computer program embedded in the blockchain, with which people interact via writing certificates.

We are talking about more than just Smart Contracts here; we are talking about **decentralized applications (DApps)** because this system allows for the automatic management of principles and rules on a completely decentralized network while remaining secure through the mechanisms of the blockchain.

To conclude this last tutorial and the course on Blockchain and Applications, I propose you use your new tool to code a small decentralized application.

Form a group of 4 people with others of your choice. Once your group is formed, come see me (during the course, on Teams, or by email) presenting the composition of your group. I will then give you a decentralized application topic that you must create using everything we have coded during the module, including this assignment. Your topic will include the following elements:
* The objective of the application, in the broadest sense of the term
* A more detailed description of all the functionalities that your DApp must present
* A set of aids on the realization of this application

**It is mandatory to implement the DApp using the blockchain system we implemented throughout the module.** You can, however, modify it any way you want to meet the requirements for the DApp (add new certificate types, change the smart contract system, etc...).

With your code, you are required to **write a notebook** that showcases a demonstration of your DApp through a mini-scenario. Simulate users interacting with your application and show how things go.

On the day of the last course (February 28th), you will have to orally present your notebook as well as your code and the choices you made in the design of the application.

I remind you that this mini-project counts for 20% of the final grade.

***

Whatever the project you receive, you will need to write your contract in a `.py` file as if you were going to use it conventionally, unlike in this notebook where the mini-chat is written in a cell. Do not panic, this changes nothing since you store the source code in the blockchain in the form of a string. You will simply need to load the `.py` file as if it were a text file.

To do this, here is a function that returns, as a string, the entire content of a **file located in the same folder as this notebook**.

In [109]:
def read_text_file(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()